# ENGR-E 533: Deep Learning Systems
## Homework 1

### Khandokar Md. Nayem (knayem@iu.edu)
### Mar 7, 2018

### Import necessary files and set environment parameters
My assigned Node is `r-006` and GPU `1`.

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
import numpy as np
import librosa

### Importing MNIST dataset

In [ ]:
s, sr=librosa.load('train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)

sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)

print(S.shape,X.shape)

In [ ]:
S_mag = np.abs(S)
X_mag = np.abs(X)
print(S_mag.shape,X_mag.shape)

In [ ]:
NUM_ITERATION = 1000

### Xavier Initialization of Weights
These are the weight initialization function used in defining model.

In [ ]:
def weight_variable (shape):
    initial = tf.truncated_normal(shape, stddev = np.sqrt(2.0/sum(shape)) )
    return tf.Variable(initial)

def bias_variable (shape):
    initial = tf.truncated_normal(shape, stddev = np.sqrt(1.0/sum(shape)) )
    return tf.Variable(initial)

### Create the fully connected model 

In [ ]:
x = tf.placeholder(tf.float32, [None, 513]) 

W_1 = weight_variable([513, 1024])
b_1 = bias_variable([1024])

W_2 = weight_variable([1024, 1024])
b_2 = bias_variable([1024])

W_3 = weight_variable([1024, 1024])
b_3 = bias_variable([1024])

W_4 = weight_variable([1024, 1024])
b_4 = bias_variable([1024])

W_5 = weight_variable([1024, 513])
b_5 = bias_variable([513])

y_ = tf.placeholder(tf.float32, [None, 513]) # original


# Layer connections and Activation functions
y_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)
y_2 = tf.nn.relu(tf.matmul(y_1, W_2) + b_2)
y_3 = tf.nn.relu(tf.matmul(y_2, W_3) + b_3)
y_4 = tf.nn.relu(tf.matmul(y_3, W_4) + b_4)
y =  tf.nn.relu(tf.matmul(y_4, W_5) + b_5) # predicted


# Define loss and optimizer
mse = tf.reduce_sum( tf.losses.mean_squared_error(labels=y_, predictions=y) )
train_step = tf.train.AdamOptimizer().minimize(mse)

### Run the model
If we increase the number of iteration (Line 10, currently 1000), `Accuracy` increases; but more time is needed to train.

In [ ]:
# Configuration to control GPU use
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33
sess = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()


# Train Model
for _ in range(NUM_ITERATION):
    sess.run(train_step, feed_dict={x: X_mag.T, y_: S_mag.T})

In [ ]:
# Load Test data
sn, sr=librosa.load('test_x_01.wav', sr=None)
X_test_01=librosa.stft(sn, n_fft=1024, hop_length=512)
X_mag_test_01 = np.abs(X_test_01)

sn, sr=librosa.load('test_x_02.wav', sr=None)
X_test_02=librosa.stft(sn, n_fft=1024, hop_length=512)
X_mag_test_02 = np.abs(X_test_02)

print(X_mag_test_01.shape)

In [ ]:
# Test trained model
S_hat_mag_test_01=sess.run(y, feed_dict={x: X_mag_test_01.T})
S_hat_mag_test_02=sess.run(y, feed_dict={x: X_mag_test_02.T})

S_hat_test_01=(X_test_01/X_mag_test_01)*S_hat_mag_test_01.T
S_hat_test_02=(X_test_02/X_mag_test_02)*S_hat_mag_test_02.T

S_hat_01=librosa.istft(S_hat_test_01, hop_length=512)
S_hat_02=librosa.istft(S_hat_test_02, hop_length=512)

librosa.output.write_wav('test_s_01_recons.wav', S_hat_01, sr)
librosa.output.write_wav('test_s_02_recons.wav', S_hat_02, sr)